## Can we write a Sonnet like its the middle ages?

![sonnet](Sonnet_art.jpg)

### 1.0 Loading the data
<hr/>
Let's start with loading the actual data and seeing a small sample of it, say the first 300 characters.

In [1]:
data = open('E:\CODE\github-repos\poetry-generator\src\Sonnets\data\sonnets.txt').read()
data[0:300]

"FROM fairest creatures we desire increase,\nThat thereby beauty's rose might never die,\nBut as the riper should by time decease,\nHis tender heir might bear his memory:\nBut thou, contracted to thine own bright eyes,\nFeed'st thy light'st flame with self-substantial fuel,\nMaking a famine where abundance"

### 2.0 Load the packages
<hr/>
Remeber to check the requirements file for packages used.

In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers 
import tensorflow.keras.utils as ku 
import numpy as np 

Checking the tenorflow version

In [3]:
import tensorflow as tf
print(tf.__version__)

2.1.0


### 3.0 Tokenizing the traing data
<hr/>

In [4]:
tokenizer = Tokenizer()
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
print('Total number of words in corpus:',total_words)

Total number of words in corpus: 3211


### 4.0 Preparing the data for training:
<hr/>

This is the most important part of this entire script and can be broadly split into 5 steps. So let's get into it shall we,
 #### 4.1) Converting text to sequences.
 #### 4.2) Creating the N_gram sequences.
 #### 4.3) Finding the max sequence length and the padding the rest.
 #### 4.4) Creating the predictors and the labels.